# Unit Testing and Debugging
**Author: Zak Hussain**  
**Date: 4/18/2019**  

This tutorial introduces you to building blocks for unit testing in Q#. Diagnosing errors and understanding the expected outcome of your code is as true in quantum computing as it is in classical computing. 

In this notebook, you will see how the Q# language and built-in libraries allow you to build/use operations and functions to test your code.

Before starting this tutorial, I recommend you complete the following notebooks:  
* [Introduction to the Qubit](../Qubit/Qubit.ipynb)
* [Single Qubit Gates](../SingleQubitGates/SingleQubitGates.ipynb) 
* [Multiqubit Gystems](../MultiQubitSystems/MultiQubitSystems.ipynb)
* [Multiqubit Gates](../MultiQubitGates/MultiQubitGates.ipynb)

For each topic, I will link you to additional readings found in the microsoft QDK docs. I will also state what you should take away from the assigned reading. 

Given that you've completed the four tutorials above, you should understand how gates are used to manipulate qubit registers. With that, you can focus on navigating the Q# language to start writing unit tests.

We will cover the following: 

* Operations and functions 
* Using the diagnostics namespace 
* Raising errors using fail and facts
* Dump Functions

At the end of the notebook, there are exercises in building unit tests. they will start relatively simple (given you understand the fundamentals of Q#) and gradually get more difficult. Spend at least 20 minutes on each problem before looking at the answers. 

For your use, here is a [cheatsheet for Q#](https://github.com/microsoft/QuantumKatas/blob/master/quickref/qsharp-quick-reference.pdf).

To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the exercises):

In [1]:
%package Microsoft.Quantum.Katas::0.10.1911.1607

Microsoft.Quantum.Standard::0.10.1911.1607, Microsoft.Quantum.Katas::0.10.1911.1607

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


## Operations and Functions  

For this section, access this [article](https://docs.microsoft.com/en-us/quantum/techniques/operations-and-functions) and read the following sections:  
* [Defining New Operations](https://docs.microsoft.com/en-us/quantum/techniques/operations-and-functions#defining-new-operations)
* [Defining New Functions](https://docs.microsoft.com/en-us/quantum/techniques/operations-and-functions#defining-new-functions)

From the sections above, you should answer the following:  
* What is an operation? 
* What is a function used for in Q#? 
* how does an operation differ from a function? 
* Why is Q# called a "tuple-in-tuple-out" langauge?  

As described in the reading, **operations** are used to carry out manipulations to quantum information; whereas, **Functions** are used solely for classical computation. 

Your Q# programs will largely consist of operations that act on qubit registers. Operations are composed of other operations and can call functions. It is important to note that functions do not act on qubits and cannot call operations. They are used for computing classical values (e.g., 1+1, 2x2). Later, we will see that functions have a characteristic that make them useful for embedding into assertions operations.

Let's start with looking at an example operation.  

We will use the BitFlip operation, from the "Defining New Operations" section of the reading, to observe how we can construct a unit test operation. 

### <span style="color:blue">Example 1</span>: The BitFlip Operation

**Input:** A qubit in an arbitrary state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$.

**Goal:** Apply the $X$ gate to the qubit (i.e., transform the given state into $|1\rangle$ if the target qubit starts in the $|0\rangle$ state and vice versa).

In [2]:
operation BitFlip(target : Qubit) : Unit {
    X(target); 
}

BitFlip

The BitFlip operation flips the state of an input qubit. This operation's output is the empty tuple (), as defined by the output type *Unit*. The body of our code calls the built-in Q# operation [*X*](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.x), which flips our target qubit.

Testing this BitFlip operation requires us to do the following:  
1. Define an test operation that will call the BitFlip operation.
2. Allocate a qubit to store our quantum state. 
3. Check that BitFlip manipulates our quantum state correctly. 

To accomplish step 2, we can use the [*Assert*](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.assert) or [*AssertProb*](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.assertprob) operation found in the [Intrinsic](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic) namespace. In addition, using those two operations requires us to run [*Pauli measurements*](https://docs.microsoft.com/en-us/quantum/concepts/pauli-measurements) (e.g., PauliX, PauliZ) as demonstrated in the code block below. 

TestBitFlipUsingAssert works as follows:  
1. Allocates a single qubit. 
2. Executes the BitFlip operation on the qubit. 
3. Runs the Assert on our qubit based on the PauliZ measurement. 
4. Resets the qubit back to its $|0\rangle$ state. 

In [3]:
// run this code block, 
// then excecute this operation by running simulate cell below.
operation TestBitFlipUsingAssert () : Unit {
    // allocate a qubit in the |0> state.
    using (qubit = Qubit()) 
    {   
        // flip the qubit from the |0> state to the |1> state
        BitFlip(qubit); 
        
        // ensure that the qubit results in the One state, given the effect of the PauliZ measurement.
        // TODO: after you successfuly simulated this operation by running the cell below, try changing the One to Zero.
        Assert([PauliZ], [qubit], One, "The qubit did not result in the One state given the PauliZ effect."); 
        
        // Release the qubit
        Reset(qubit); 
    }
}

TestBitFlipUsingAssert

In [4]:
%simulate TestBitFlipUsingAssert

()

Notice that we get an empty tuple after running the simulation cell above. This is good! it means our test passed. We are using an Assert that applies the PauliZ measurement effect on the qubit and we expect to get the One Result type. 

Now, go back to the Assert operation and try changing the One result to a Zero as marked in the TODO comment. Once you've made the change, re-run the two code cells, and you should now see an error message! 

Let's answer the question of how our Assert operation is being used in Q#. We pass in a Pauli operator that describes a measurment of interest, the quantum register which we perform the measurement, the hypothetical outcome, and an error message. Since we are executing our code in simulation, the measurement we perform does not manipulate our register. Once our  programe is run, if the condition is found false, the Assert will raises a *fail* alongside our error message. 

We will see an operation clearly showing that assertions do not manipulating our register in Example 2. Also, read about the [Pauli Measurements](https://docs.microsoft.com/en-us/quantum/concepts/pauli-measurements) as they are fairly useful for identifying quantum states. We used the PauliZ measurement in the Assert operation above, and now we will use the PauliX in Example 2. 

### <span style="color:blue">Example 2</span>: The H Gate

**Input:** A qubit in an arbitrary state $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$.

**Goal:** Apply the $H$ gate to the qubit (i.e., create a superposition)

In [5]:
operation CreateSuperposition (q: Qubit) : Unit {
        // place the qubit in superposition, (|0> + |1>)/sqrt(2). 
        H(q); 
}

CreateSuperposition

Given the above operation, we can define a test operation, TestSuperpositionUsingAssertProb, that will accomplish the following: 
1. Allocate a qubit. 
2. Execute the CreateSuperposition operation on our qubit.
3. Use AssertProb to ensure the correctness of CreateSuperposition by measuring the PauliX effect on the qubit.
4. Set the qubit back to it's original state.

In [6]:
operation TestSuperpositionUsingAssertProb () : Unit {
    using (qubit = Qubit()) {
        // place qubit in superposition. 
        CreateSuperposition(qubit); 
        
        // assert that the PauliX effect on the qubit in superposition will result in the |0> state with a 
        // probability success of 1.0 within a 0.0 tolerance.
        AssertProb([PauliX], [qubit], Zero, 1.0, "The qubit does not result in the Zero state given the PauliX effect.", 0.0);
        
        // release the qubits 
        Reset(qubit); 
    }
}

TestSuperpositionUsingAssertProb

In [7]:
%simulate TestSuperpositionUsingAssertProb

()

Given that you ran the above cell successfully, try changing the Result state of the AssertProb operation from Zero to One. Why do you think this triggers the failure message?  

To show that the assertion does not change the state of our register, we can use the DumpMachine to observe the state of our qubit(s). Using the same steps as in TestSuperpositionUsingAssertProb, let's create an operation DumpSuperpositionUsingAssertProb that will also dump the state of our qubit after the assertion. 

It will work as follows:  

1. Allocate a qubit.
2. Execute the CreateSuperposition operation on our qubit.
3. Use AssertProb to ensure the correctness of CreateSuperposition by measuring the PauliX effect on the qubit.
4. Use DumpMachine to observe the state of our quantum system.
5. Set the qubit back to it's original state.

In [8]:
// open the diagnositcs namespace in this cell to access the DumpMachine
open Microsoft.Quantum.Diagnostics; 

operation DumpSuperpositionUsingAssertProb () : Unit {
    using (qubit = Qubit()) {
        // place qubit in superposition. 
        CreateSuperposition(qubit);  
        
        // assert that the PauliX effect on the qubit in superposition will result in the |0> state with a 
        // probability success of 1.0 within a 0.0 tolerance.
        AssertProb([PauliX], [qubit], Zero, 1.0, "The qubit does not result in the Zero state given the PauliX effect.", 0.0);
        
        // Use DumpMachine to look at the state of our entire system 
        DumpMachine(); 
        Reset(qubit); 
    }
}

DumpSuperpositionUsingAssertProb

In [9]:
%simulate DumpSuperpositionUsingAssertProb

# wave function for qubits with ids (least to most significant): 0
∣0❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]
∣1❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]


()

Given that we have seen a couple assert opertations, lets look at the Diagnostics namespace, which provides us other operations and functions we have available to diagnose our code. 

## Getting Started with Diagnostics    

In the previous section we used the *Assert* and *AssertProb* operations from the Intrinsic namespace. We used these assertions for checking the state of our qubit. The Diagnostics namespace furthers our ability to debug errors. It is composed of additional assertion operations and functions. With this, you will be able to compare operations and characterize the state of your qubits. 

For this section, access this [Diagnostics article](https://docs.microsoft.com/en-us/quantum/libraries/standard/diagnostics) and read the following sections:  
* Testing Qubit States
* Asserting Equality of Quantum Operations 

From the reading listed above, you should answer the following:  
* Do assertions affect the state of our quantum states? 
* How do we handle qubit states that do not coincide with eignestates of Pauli operators? What type of Assertion works for this problem? 
* What is an Adjoint operation? 

Let's jump right into looking at an example of the AssertQubit operation mentioned in the reading. 

### <span style="color:blue">Example 3</span>: The AssertQubit Operation

**Input:** A qubit in state $|\psi\rangle = |0\rangle$

**Goal:** Apply the $Y$ gate to the qubit, i.e., transform the given state into $|\psi\rangle = i|1\rangle$.

In [10]:
// operation to test notice the return type indicates the operation specializations--adjoint and controlled.
operation ApplyY (q : Qubit) : Unit is Adj+Ctl {
    Y(q); 
}

ApplyY

We can create a Test operation for ApplyY that will do the following: 
1. Allocate a single qubit in the $|0\rangle$ state. 
2. Run the ApplyY operation on our qubit. 
3. Assert the state of the qubit qiven the PauliZ operator. 
4. Write a success Message. If this line runs, we know our Assert did not fail. 
5. Reset the qubit into the |0> state. 

In [11]:
// open namespace containing the AssertQubit operation and the Message()
open Microsoft.Quantum.Diagnostics;

operation TestApplyY () : Unit {
    // allocate a qubit in the |0> state
    using (qubit = Qubit()) {
        
        // apply the Y operation.
        ApplyY(qubit); 
        
        // Assert the qubit is in the expected eigenstate of the Pauli Z operator. 
        AssertQubit(One, qubit); 
        
        // display message once we execute the simulation cell.
        Message("The TestApplyY operation ran successfully.");
        
        // release the qubit. 
        Reset(qubit); 
    }
}

TestApplyY

In [12]:
%simulate TestApplyY

The TestApplyY operation ran successfully.


()

Now try triggering the AssertQubit fail message. 
><details><summary><u>Click here for a hint</u></summary>
> Change the Result input from One to Zero.  
> The effect of the Pauli Z measurment on a qubit in an $i|1\rangle$ state is a qubit in the -1 eigenstate of Z, which is the Result, One. 
> </details>

With testing qubit states, there are often cases where assertions that coincide with the eigenstates of Pauli operatators do not work. The example in the *Testing Qubit States* section of the  [Diagnostics article](https://docs.microsoft.com/en-us/quantum/libraries/standard/diagnostics) gives the example state $|\psi\rangle = (|0\rangle + e^{\frac{i\pi}{8}}|1\rangle)/\sqrt{2}$. It then discusses the process of identifying three independent assertions to examine the Result x,y,z values for the Pauli X, Y, and Z measurements, respectively. Assuming you have read this section, let's look at an example of testing using the [**AssertQubitIsWithinTolerance**](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.assertqubitisinstatewithintolerance) operation, which implements the three independent probability equations below. Note that $\alpha$ and $\beta$ are both [Complex](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.math.complex) types. 
* $P(x=Zero|\alpha, \beta)$
* $P(y=Zero|\alpha, \beta)$
* $P(z=Zero|\alpha, \beta)$

### <span style="color:blue">Example 4</span>: The AssertQubitWithinTolerance Operation

**Note:** The following links provide info on components used in this example: 
* [Pi Function](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.math.pi)  
* [R1 Operation](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.r1)  
* [Complex Type](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.math.complex) 

**Input:** A qubit in the state $|\psi\rangle = |0\rangle$

**Goal:** create the state $|\psi\rangle = \frac{1}{\sqrt{2}}(|0\rangle + e^{\frac{i\pi}{8}}|1\rangle)$

In [13]:
// open Math namespace to access the PI() function 
open Microsoft.Quantum.Math;

// operation to test
operation CreateNonPauliEigenstate (qubit : Qubit) : Unit {
    // create a superpostion 
    H(qubit); 
    
    // apply a rotation about the |1> state by the given theta. 
    let theta = PI()/8.0; 
    
    // use the r1 operation to apply (e^i*theta) to the |1> state. 
    R1(theta, qubit); 
}

CreateNonPauliEigenstate

We can create a Test operation for CreateNonPauliEigenstate that will do the following: 
1. Allocate a single qubit in the $|0\rangle$ state. 
2. Run the CreateNonPauliEigenstate operation on our qubit. 
3. Assert the state of the qubit is correct given its Complex $\alpha$ and $\beta$ values. 
4. Write a success Message. If this line runs, we know our assert did not fail. 
5. *optional* uncomment the DumpMachine line to see the state of the entire system.
6. Reset the qubit into the $|0\rangle$ state. 

*Note: Hopefully, you are starting to see a pattern in building basic tests.*

In [14]:
// use the Math namespace to access the 'Complex' type. 
open Microsoft.Quantum.Math;
// to see the resulting complex states for the qubit, give access to the DumpMachine function. 
open Microsoft.Quantum.Diagnostics;

operation TestCreateNonPauliEigenstate () : Unit {
    
    // allocate a qubit in the |0> state
    using (qubit = Qubit()) {
        
        // execute the operation we want to test. 
        CreateNonPauliEigenstate(qubit); 
        
        // check that we correctly put the qubit in the (|0⟩+𝑒^𝑖𝜋/8|1⟩)/sqrt(2) state.
        // R1(pi/8)((|0〉 + |1>)/sqrt(2)) = (|0⟩+𝑒^𝑖𝜋/8|1⟩)/sqrt(2) : a=(0.707 + 0i), b=(0.653 + 0.271i)
        AssertQubitIsInStateWithinTolerance((Complex(0.707, 0.), Complex(0.653, 0.271)), qubit, 1e-3);
        
        Message("The AssertQubitIsInStateWithinTolerance operation ran successfully"); 
        
        // uncomment the DumpMachine to see the complex probability amplitudes for each state. 
        //DumpMachine();
        
        // release qubit.
        Reset(qubit); 
    }
}

TestCreateNonPauliEigenstate

In [15]:
%simulate TestCreateNonPauliEigenstate

The AssertQubitIsInStateWithinTolerance operation ran successfully


()

Try changing the tolerance component in the testing operation from 1e-3 to a smaller tolerance, e.g., 1e-5. Why do you think this causes the Assertion to raise the failure message? 

We've taken a look at using assertions to look at the states of our qubits; however, we fairly often use assertion to compare operations. For example, we may want to know if two operations act the same way on a qubit register, and we want to identify the operation with the least compute. In this case, if we prove they act the identically, we just have to identify which of the two are more optimal.  

Let's now take a look at the process involved in test the likeness of operations. 

### <span style="color:blue">Example 5</span>: The AssertOperationEqualInPlaceCompBases Operation

We will first create two operations, CreateBellStateWithCNOT and CreateBellStateWithControlledX, given the following:  

**Input:** Two qubits in state $|00\rangle$

**Goal:** Transform the system into the Bell state $\Phi^+ = \frac{1}{\sqrt{2}}\big(|00\rangle + |11\rangle\big)$.

In [16]:
// operation 1
operation CreateBellStateWithCNOT (qubits : Qubit[]) : Unit is Adj+Ctl{
    H(qubits[0]); 
    CNOT(qubits[0], qubits[1]);
}

CreateBellStateWithCNOT

In [17]:
// operation 2
operation CreateBellStateWithControlledX (qubits : Qubit[]) : Unit is Adj+Ctl {
    H(qubits[0]); 
    Controlled X([qubits[0]], qubits[1]);
}

CreateBellStateWithControlledX

We will use the following process to test the operations based on how we use [AssertOperationsEqualReferenced]( https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.assertoperationsequalreferenced):  
1. Declare the number of qubits. 
2. Declare the two operations we want to compare.
3. Execute the assertion with the components from part 1 and 2. 
4. Display a success message if the assertion doesn't fail.

In [18]:
// access the assertion operation used in this cell 
open Microsoft.Quantum.Diagnostics;

operation TestOperationsAreEqual () : Unit {
    let nQubits = 2; 
    let givenU = CreateBellStateWithControlledX;
    let expectedU = CreateBellStateWithCNOT; 
    
    AssertOperationsEqualReferenced(nQubits, givenU, expectedU); 
    Message("The AssertOperationsEqualReferenced operation ran successfully, the operations act identically.");
}

TestOperationsAreEqual

In [19]:
%simulate TestOperationsAreEqual

The AssertOperationsEqualReferenced operation ran successfully, the operations act identically.


()

What would have happened if we changed the CreateBellStateWithControlledX operation to contain Controlled X([qubits[1]], qubits[0])? 

> <details> 
> <summary><u>Click here to see answer</u></summary>
> This would raise the exception because this It would fail to create the correct state on the testing qubits for the  
> operations to be considered equal. 
> </details>

The full list of assertion operations in the Diagnostics namespace can be found [here](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics#operations).  
With assertions covered, Let's take a look at a few other tricks we can use.

## Testing with Fact Functions  

Aside from the assertions discussed above, The Diagnostics namespace provides conditional checks in the form **facts**. A fact is a type of function that can be represented in the following forms:  
* [Fact](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.fact) 
* [EqualityWithinToleranceFact](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.equalitywithintolerancefact)
* [NearEqualityFactC](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.nearequalityfactc)
* [EqualityFactI](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.equalityfacti)

Where we saw assertions were useful for checking the states of out quantums systems, facts check the correctness of classical conditions. If the condition of a fact is false, then our program is halted, and a fail is raised. In the example below, we will look at raising a fail using a function. Following we will use one of the facts to accomplish the same task. 

### <span style="color:blue">Example 6</span>: Raising A Failure

**Input:** A register contain some number of qubits. 

**Goal:** raise fail if the register contains more than 5 qubits.

In [20]:
// open the Core namespace to access the Length function
open Microsoft.Quantum.Core;

function CheckRegisterSize(register : Qubit[]) : Unit {
    // define the max size of the register allowed.
    let maxRegisterSize = 5; 
    
    // use the Length function to get the size of the register. 
    if (Length(register) > maxRegisterSize) {
        fail "The qubit register is too large, the register should contain no more than 5 qubits."; 
    }
}

CheckRegisterSize

We will create a operation SimulateCheckRegisterSize that will call the CheckRegisterSize function using the following steps:  
1. Set a variable indicating the number of qubits we want to simulate.
2. Allocate those qubits. 
3. Call the function and give it the register. 
4. Release the qubits.

In [21]:
operation SimulateCheckRegisterSize () : Unit {
    
    // TODO: try changing this value to an integer greater than 5 to trigger the 
    // fail message.
    let numQubits = 5; 
    
    // allocate the qubit register containing numQubits. 
    using (qubitRegister = Qubit[numQubits]) {
        
        // call the function to check ensure the size of our register contains < than 5 qubits.
        CheckRegisterSize(qubitRegister); 
        
        // release the qubits. 
        ResetAll(qubitRegister); 
    }
}

SimulateCheckRegisterSize

Run the simulate cell to execute the operation above. Recall the empty-tuple () means the operation executed without failure. 

In [22]:
%simulate SimulateCheckRegisterSize

()

After you have successfully ran the cell above and seen the empty-tuple () displayed, try triggering the failure by changing the value of numQubits, in the SimulateCheckRegisterSize operation, to an integer greater than 5. 

### <span style="color:blue">Example 7</span>: Using Fact

We can also apply the built-in [**Fact**](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.fact) function from the Diagnostics namespace. Below we will look at an example of how to accomplish the same task above using this functon directly within the body of an operation.

it will work in the following steps:   
1. Set the numQubits indicating the number of qubits we want to simulate.
2. Set a variable indicating the max size our register can be.
3. Allocate numQubits. 
4. Execute the Fact function to raise a fail if the condition given is false. 
5. Release all the qubits.

In [23]:
// open the Core namespace to access the Length function
open Microsoft.Quantum.Core;
// open the Diagnostics namespace to access the fact functions
open Microsoft.Quantum.Diagnostics;

operation TestRegisterSizeLimit () : Unit {
    
    // select the number of qubits we want to simulate.
    let numQubits = 5;
    // select the max size our register can be. 
    let maxRegisterSize = 5; 
    
    using (qubitRegister = Qubit[numQubits]) {
    
        // note that only if the condition is false, then the failure will occur.
        Fact(Length(qubitRegister) <= maxRegisterSize, "The qubit register is too large, the register should contain no more than 5 qubits.");
        
        // Release the qubit.
        ResetAll(qubitRegister); 
    }
}

TestRegisterSizeLimit

In [24]:
%simulate TestRegisterSizeLimit

()

Try playing with both the numQubits and maxRegisterSize values. Try answering the question: "How does the condition passed into the Fact function differ from using a regular if condition?" *Hint: read the comments in the operation body*.  

The full list of fact-type functions are found [here](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics#functions)

## Dump Functions  

At this stage, you have already been introduced to how dump functions work. The *DumpMachine*, an example of a dump function, was introduced in the Qubit tutorial. The key point is Q# Diagnostics namespace helps us troubleshoot our quantum programs by offering two functions, [**DumpMachine**](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.dumpmachine) and [**DumpRegister**](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.diagnostics.dumpregister). These two functions dump the current state of our system into a file. 

Here we will look at an example of the DumpRegister. The structure of the information written to the dump is identical to that seen with the DumpMachine ([see the Qubit tutorial section on DumpMachine](../Qubit/Qubit.ipynb)). The difference is, DumpRegister allows us to selectively choose which qubits are placed in the dump.  Let's take a look at an example. 

### <span style="color:blue">Example 8</span>: The DumpRegister

**Input:** A qubit register containing two qubits in the |00> state. 

**Goal:** Create the seperable state, $|\psi\rangle = \frac{1}{2}(|00\rangle + i|01\rangle + i|10\rangle + |11\rangle$.

**NOTE:** For this example, navigate to your "C:" folder and create a folder called "Quantum_DumpRegister".

In [25]:
open Microsoft.Quantum.Diagnostics; 

// TODO: Create example using DumpRegister
operation CreateSeperableState () : Unit {

    // allocate two qubits
    using (qubits = Qubit[2]) {
    
        // create the desired seperable state. 
        ApplyToEachA(H, qubits); 
        ApplyToEachA(S, qubits); 
        
       // identify the location to store the qubit#.txt files.
       // manually navigate to your C folder and create the Quantum_DumpRegister folder
       let filepath_q1q2 = "C:\Quantum_DumpRegister\q1q2.txt";
       let filepath_q1 = "C:\Quantum_DumpRegister\qubit1.txt";
       let filepath_q2 = "C:\Quantum_DumpRegister\qubit2.txt";
       
       // pass the location to save the file, and the qubit(s) you want to observe.
       // note that arrays are inclusive of the max index given. 
       // in this case, we passed in both the 0th and 1st element in the register.
       DumpRegister(filepath_q1q2, qubits[0 .. 1]); 
       
       // check the state of just the first qubit. 
       DumpRegister(filepath_q1, [qubits[0]]); 
       
       // check the state of just the second qubit. 
       DumpRegister(filepath_q2, [qubits[1]]); 
       
       // release all the qubits.
       ResetAll(qubits); 
    }
}

CreateSeperableState

In [26]:
%simulate CreateSeperableState

()

Once you've run the simulation cell above, navigate to the "C:\Quantum_DumpRegister\" folder. You will see the following: 

* that we have written the entire state of the two qubits to the q1q2.txt.
* the state of just qubit one to the q1.txt.  
* the state of qubit2 to the q2.txt.   

These files visually prove we correctly created the desired state. The tensor product of qubit one and qubit two give the result shown in q1q2.txt. In the case that the qubits are entangled, DumpRegister outputs a message stating the qubit(s) you are interested is entangled with another.

Congrats! Now that you've had a chance to familiarize yourself with important tools for testing operations, we can move on to the exercises!

## Exercises

The following exercises are designed to test your understanding of the concepts you've learned so far. 
In each exercise your task is to implement an operation for each Task. You will have to allocate the qubits, and if necessary, put the qubit register into a certain initial state. Also be mindful of the Requirement(s) of each task. 
Spend at least 20 minutes per exercise, before looking at the solutions at the end of each task. Good luck! 

### <span style="color:blue">Exercise 1</span>: Pauli Measurements  
**Task1:** Complete the CreatePauliState operation.

**Input:** A qubit register in a state $|00\rangle$

**Goal:** create the state $|\psi\rangle = \frac{1}{\sqrt{2}}(\alpha|00\rangle + \beta|11\rangle)$

In [50]:
operation CreatePauliState (register: Qubit[]) : Unit {
        // ..
}

CreatePauliState

> <details>
> <summary><b>Click here to see the CreatePauliState solution</b></summary>
>    
>   operation CreatePauliState (register: Qubit[]) : Unit {    
>    
>        // place the each qubit in state, (|0> + |1>)/sqrt(2).
>         ApplyToEachA(H, register);
>
>   }  
> </details>

**Task2:** complete the TestCreatePauliState operation needed to ensure you get the correct state based on what you did in Task1. 

**Requirements:** 
* This operation must check that CreatePauliState puts a multiqubit register into the correct state. 
* The Assertion operation chosen should use the Pauli Measurements. 

In [48]:
operation TestCreatePauliState () : Unit {
    // ...
}

TestCreatePauliState

In [49]:
%simulate TestCreatePauliState

()

> <details>
>    
> <summary><b>Click here to see the TestTransformThreeQubitState solution</b></summary>
>    
>   operation TestCreatePauliState () : Unit {
>
>       // allocate 2 register in the state |0>
>       using (register = Qubit[2]) {
>            // place register in superposition.            
>            CreatePauliState(register);
> 
>             // assert that the PauliX effect on the qubit in superposition will result in the |0> state with a
>             // probability success of 1.0 within a 0.0 tolerance. 
>             AssertProb([PauliX, PauliX], register, Zero, 1.0, "The qubit does not result in the Zero state given the PauliX effect.", 0.0);
>     
>            // message indicates the assertion did not fail.
>            Message("The CreatePauliState operation ran successfully");     
>    
>            // Reset all qubits to the |000> state.
>            ResetAll(register);   
>       }
>   }
> </details>    

### <span style="color:blue">Exercise 2</span>: Create a Test with Fact 

**Task1:** complete the TransformThreeQubitState operation.   

**Input:** A qubit in the state $|\psi\rangle = \frac{1}{\sqrt{2}}(|000\rangle + |100\rangle)$.

**Goal:** Transform the given state into $\frac{1}{\sqrt{2}}(i|100\rangle - i|000\rangle)$.

**Addition Requirement:** Include a Fact condition to ensure the register given does not contain more than three qubits.

In [27]:
// open the Core namespace to access the Length function
open Microsoft.Quantum.Core;
// open the diagnositcs namespace to access the Fact function
open Microsoft.Quantum.Diagnostics; 

operation TransformThreeQubitState(qubitRegister : Qubit[]) : Unit is Adj+Ctl {
   // ... 
}

TransformThreeQubitState

> <details>
> <summary><b>Click here to see the TransformThreeQubitState solution</b></summary>
>
>   operation TransformThreeQubitState(qubitRegister : Qubit[]) : Unit is Adj+Ctl {    
>
>        // if the qubitRegister has more than 3 qubits, raise the fail.
>         Fact(Length(qubitRegister) <= 3, "The Register contains more than 3 qubits.");
>                                         
>        // apply the y gate to the first qubit gets us (i|100> - i|000>)/sqrt(2)
>        Y(qubitRegister[0]);  
>
>   }  
> </details>

**Task2:** complete the TestTransformThreeQubitState operation needed to ensure you get the correct state based on what you did in Task1. 

**Requirements:** 
* This operation must check that TransformThreeQubitState puts a three qubit into the correct state. 

In [28]:
// get access to the Complex datatype
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Diagnostics;

operation TestTransformThreeQubitState () : Unit {
    // ...
}

TestTransformThreeQubitState

In [29]:
%simulate TestTransformThreeQubitState

()

> <details>
>    
> <summary><b>Click here to see the TestTransformThreeQubitState solution</b></summary>
>    
>   operation TestTransformThreeQubitState () : Unit {
>
>       // allocate 3 qubits in the |000> state
>       using (qubitRegister = Qubit[3]) {
>            // prepare the register in the state (|000> + |100>)/sqrt(2).            
>            H(qubitRegister[0]);
> 
>            // create the quantum state (i|100> - i|000>)/sqrt(2).
>            TransformThreeQubitState(qubitRegister); 
>
>            // ensure we've created the state (i|100> - i|000>)/sqrt(2).
>            AssertQubitIsInStateWithinTolerance((Complex(0., 0.707), Complex(0., -0.707)), qubitRegister[0],1e-3);  
> 
>            // message indicates the assertion did not fail.
>            Message("The TransformThreeQubitState operation ran successfully");     
>    
>            // Reset all qubits to the |000> state.
>            ResetAll(qubitRegister);   
>       }
>   }
> </details>    

**Task3:** complete the TestTestTransformThreeQubitStateFact operation to check that the Fact assertion you added in the
TransformThreeQubitState operation is raised. 

**Requirements:** 
* This operation must trigger Fact condition the TransformThreeQubitState.  Note that when you run this test, you should get a red box appearing with the error message you used in your fact function.

In [30]:
operation TestTransformThreeQubitStateFact () : Unit {
    // ...
}

TestTransformThreeQubitStateFact

In [35]:
%simulate TestTransformThreeQubitStateFact

()

> <details>
>    
> <summary><b>Click here to see the TestTransformThreeQubitState solution</b></summary>
>    
>   operation TestTestTransformThreeQubitStateFact () : Unit {
>
>       // allocate 4 qubits in the |0000> state
>       using (qubitRegister = Qubit[4]) {
>            // prepare the register in the state (|0000> + |1000>)/sqrt(2).            
>            H(qubitRegister[0]);
> 
>            // create the quantum state (i|1000> - i|0000>)/sqrt(2).
>            TransformThreeQubitState(qubitRegister); 
>
>            // message indicates the assertion did not fail.
>            Message("If you see this message, then you Fact function failed to catch the error. The qubit register contains more than 3 qubits.");     
>    
>            // Reset all qubits to the |0000> state.
>            ResetAll(qubitRegister);   
>       }
>   }
> </details>    

## Conclusion

Congratulations! Now that you have gone through this notebook on building unit tests, you should have a good idea of the tools available to you in Q# to build more complicated tests. Now, you are are neither a pro at building unit test, nor are you an amature, rather you are in a superposition of both states. Have fun explore Quantum! (:  